# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import json

# Import API key
from api_keys import geoapify_key

In [9]:
# Verifying API key
geoapify_key

'1b2e108cbc384fca90224c0d4c0b4f92'

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,zaragoza,41.6561,-0.8773,64.49,57,75,4.61,ES,1711677942
1,1,fale old settlement,-9.3852,-171.2468,85.84,74,46,15.03,TK,1711677942
2,2,port-aux-francais,-49.3500,70.2167,40.84,75,98,8.21,TF,1711677942
3,3,sovetskiy,61.3614,63.5842,12.29,92,100,10.22,RU,1711677942
4,4,adamstown,-25.0660,-130.1015,76.19,74,39,18.28,PN,1711677942


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity',
                                      hover_cols=['City', 'Humidity'],
                                      color='Humidity', cmap='blues',
                                      title="Humidity",
                                      tiles='OSM')
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 70) & 
    (city_data_df['Max Temp'] < 90) &
    (city_data_df['Cloudiness'] < 5)
]

# Drop any rows with null values
ideal_weather_df_clean = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df_clean.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,acapulco de juarez,16.8634,-99.8901,75.02,88,0,5.75,MX,1711677839
18,18,wailua homesteads,22.0669,-159.3780,81.03,68,0,19.57,US,1711677945
29,29,namibe,-15.1961,12.1522,77.52,75,4,4.70,AO,1711677947
31,31,carnarvon,-24.8667,113.6333,83.14,41,0,15.32,AU,1711677947
95,95,saint-pierre,-21.3393,55.4781,76.68,73,0,5.75,RE,1711677959


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df_clean[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
14,acapulco de juarez,MX,16.8634,-99.8901,88,
18,wailua homesteads,US,22.0669,-159.3780,68,
29,namibe,AO,-15.1961,12.1522,75,
31,carnarvon,AU,-24.8667,113.6333,41,
95,saint-pierre,RE,-21.3393,55.4781,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories' : 'accommodation.hotel',
    'limit': 1,
    'apiKey' : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionat
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
acapulco de juarez - nearest hotel: Hotel del Valle
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
namibe - nearest hotel: Nelsal Pensao
carnarvon - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
bairnsdale - nearest hotel: Main hotel
cockburn town - nearest hotel: The Salt Raker Inn
telimele - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
las vegas - nearest hotel: Downtown Grand
jacmel - nearest hotel: Hotel L'Amenitie
tadine - nearest hotel: Hôtel Nengoné
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
remire-montjoly - nearest hotel: Complexe Belova
brisas de zicatela - nearest hotel: Casa de Olas
alice springs - nearest hotel: Aurora Alice Springs
san jeronimo de juarez - nearest hotel: No hotel found
bhuban - nearest hotel: No hotel found
isla mujeres - nearest hotel: Villa La Bella
murray bridge - nearest hotel: Bridgeport
san luis de la loma - nearest hotel: No hotel found
rahimatp

,City,Country,Lat,Lng,Humidity,Hotel Name
14,acapulco de juarez,MX,16.8634,-99.8901,88,Hotel del Valle
18,wailua homesteads,US,22.0669,-159.3780,68,Hilton Garden Inn Kauai Wailua Bay
29,namibe,AO,-15.1961,12.1522,75,Nelsal Pensao
31,carnarvon,AU,-24.8667,113.6333,41,No hotel found
95,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, 
                              tiles='OSM',
                              hover_cols=['City', 'Hotel Name', 'Country'],
                              color='Humidity',
                              size=10,
                              alpha=0.5,
                              title="Hotels in Ideal Weather Cities")
# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)